<h1> SGWB Parameter Space Analysys</h1> 

In the following, we'll implement a notebook that, given the required Probability Distribution Functions (PDF) describing a Black Hole population(BH), generates the figure of merit for the predicted analytical Stochastic Gravitational Wave Background(SGWB) in function of the amplitude and redshift range of the merging rate.
First of all, we need to import some modules ! 

In [1]:
import numpy as np
import scipy.special as sc
import statistics as st
import random
import os
import IPython
import pandas as pd
import pickle
import multiprocessing as mp
import scipy.stats as scst
#from tqdm import tqdm
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.integrate import quad, simpson, dblquad
from scipy.stats import poisson
from scipy.special import gamma, hyp1f1
from scipy.optimize import minimize
from pycbc import waveform as wf
from multiprocessing import Pool, Manager, Process, Value, Array
from functools import partial
#from LISAhdf5 import LISAhdf5,ParsUnits
#%matplotlib inline
#import matplotlib.pyplot as plt
#plt.style.use("seaborn-v0_8-whitegrid")

<h2> Global Variables of the Simulation </h2>

The global variables of the simulation will be set to :

In [2]:
# Flags for the execution modes, initialized to false, check the the FLAG selection section for additional informations and initializing them !

PBH_LogNormal = False
PBH_Gaussian = False

# Merger distribution parameters

T_obs = 10. # Lisa or LIGO estimated years of observation
efficiency = 1. # Lisa effective usefull time percentage for observations
max_tc = 10000. # max years of coalescence time for a BBH mergine event
frq_min = 3.e-5 # Hertz
frq_max = 0.5 # Maximum frequency in hertz to which the LISA detector is sensitive
frq_star = 1.e-2 # Value of the choosen frequency at which we estimate the SGWB to compare with other results
# The total time used to generate the merging events by multipling for the rate of merging will be set to max_tc


#General Constants 

c = 299792.46 # speed of light in Km/sec
G = 6.674*(10.**(-11.)) # Gravitational constant in m^3⋅kg^−1⋅s^−2
sol_mass = 1.988e30 # Value of the Solar Mass in Kg
MPc = 3.08567758149137*1e22 # meters
GPc = MPc*1e3 # meters
sig_M = 0.004 # rescaled variance of matter density perturbations
h = 0.678
H_0 = 67.8*1e3/MPc # Hubble constant in 1/(s)
Omega_m = 0.3 # Matter density in our universe
Omega_lambda = 0.7 # Cosmological constant density in our universe
Omega_k = 0. # Curvature density in our universe
rho_c = (3.*(H_0**2.))/(8.*np.pi*G) # Critical density in our universe
year = 365.25*24*60*60 # Years in second 
    
# Precision settings for the binned variables

n_jobs = 80
frq_res = 1e-6
frq_prec = int((frq_max - frq_min)/frq_res) + 1


<h2> FLAG selection section </h2>

We have to choose between the two following different mass functions :

In [3]:
#PBH_LogNormal = True# This will simulate the Log Normal mass distribution for PBH described in ariv 2109.05836
PBH_Gaussian = True # This will simulate a Gaussian PBH mass distribution, that can be used to generalize a bit the standard monocromatic mass function for PBH

<h2> Utility functions </h2>

In the following, we are going to define some useful generical functions that will be needed to present the results.
We will start with a function that can be used to convert matplotlib contour line to arrays.

In [4]:
def get_contour_verts(cn):
    # Given a set of contour line, save them as a dictionary
    contours = []
    # for each contour line
    for cc in cn.collections:
        paths = []
        # for each separate section of the contour line
        for pp in cc.get_paths():
            xy = []
            # for each segment of that section
            for vv in pp.iter_segments():
                xy.append(vv[0])
            paths.append(np.vstack(xy))
        contours.append(paths)

    return contours

<h2> Standard Cosmological Functions </h2>

First of all, we'll need a function that allow us to convert from redshift to Gigaparsec :

In [5]:
# Just a function to convert from Z to GPC using Hubble Law, in order to obtain the comoving distance

z_max = 1.e8
z_prec = 10000

def H(z):
    return np.sqrt((H_0**2.)*(Omega_m*((1. + z)**3.) + Omega_k*((1. + z)**2.) + Omega_lambda))

def Z_to_Gpc(z):
    
    # Remove the commented part to use a linear approximation of the Hubble law for low z 
    
    #if(zmax <= 0.5):
    #    return ((z*c*(10**(-3)))/(H_0)) # only valid for z < 0.5
    #else:
        
        span_z = np.linspace(0.,z,z_prec)
        span_mz = 0.5*(span_z[1::] + span_z[:-1:])
        
        # Beware, would fail if the span z is created in logarithmic scale !
        
        Int_Z = c*(10**(-3))*simpson(1./(H(span_mz)*(MPc/1.e3)), span_mz, axis=0)
    
        return Int_Z
    
def Z_to_HubbleTime(z):
    
    span_z = np.logspace(np.log10(z),np.log10(z_max),z_prec)
    span_mz = 0.5*(span_z[1::] + span_z[:-1:])
        
    # Beware, would fail if the span z is created in logarithmic scale !
        
    
    Int_Z = simpson(1./(H(span_mz)*(1. + span_mz)), span_mz, axis=0)
    
    return Int_Z
    
t_0 = Z_to_HubbleTime(1.e-12) # Can't put 0 as the logarithmic scale would fail        

we also need a function that estimates the differential comoving volume in function of the redshift :

In [6]:
#In the following function, the differential comoving volume in function of the redshift will be estimated as a spherical surface, it need to be integrated over dr to obtain the real volume 

def DeVC(z, Delta_z):
    r = dist_func(z)
    z_2 = z + 0.5*Delta_z
    z_1 = z_2 - Delta_z
    Delta_r = dist_func(z_2) - dist_func(z_1)
    return ((4.*np.pi*(r**2.)*Delta_r)/Delta_z)

Another recurring parameter for inspiralling events is the Chirp Mass and Reduced Mass, given the mass of the two events involved in the binary merging :

In [7]:
# Function that return the Chirp Mass of a binary merging event

def ChirpMass(m1,m2): 
   return ((m1*m2)**(3./5.))/((m1+m2)**(1./5.))

In [8]:
def EtaMass(m1,m2):
    return (m1*m2)/((m1 + m2)**2.)

In [9]:
def S1(f_PBH, m1, m2, avg_m, avg_m2):
    res = 1.42*((avg_m2/(avg_m**2.))/(N_MinDist(f_PBH, m1, m2, avg_m) + C_S1(f_PBH, avg_m, avg_m2)) + (sig_M**2.)/(f_PBH**2.))**(-21./74.) 
    return (res*np.exp(-N_MinDist(f_PBH, m1, m2, avg_m)))

In [10]:
def N_MinDist(f_PBH, m1, m2, avg_m):
    # Return the minimal distance of the third PBH required for smoothing
    res = ((m1 + m2)/avg_m)*(f_PBH/(f_PBH + sig_M))
    return res

In [11]:
def C_S1(f_PBH, avg_m, avg_m2):
    res = ((f_PBH**2.)*(avg_m2/(avg_m + sig_M)**2.))/( ((gamma(29./37.)/np.sqrt(np.pi))*hyp1f1(21./74., 0.5, (5.*f_PBH**2.)/(6.*sig_M**2.)))**(-74./21.) - 1.)
    return res

<h3> SOBBH - Redshift dependent statistic </h3>

We may now define, the various implemented merging rates as a function of the redshift _z_ as :

In [12]:
# Function for the merging rate as described in the paper arxiv 2010.14533, the flag Red_evol will decide if adopting a merging rate the evolve with redshift (true) or not (false)


SOBBH_k = 2.9 # + 1.7 - 1.8  VALID FOR REDSHIFT EVOLVING POWER LAW + PEAK MODEL MASS DISTRIBUTION, total agreement with SFR
SOBBH_CorrRz = (((1. + 0.2)**SOBBH_k)/(1. + ((1. + 0.2)/2.9)**(SOBBH_k + 2.9)))**(-1) # Normalization factor estimated at z = 0.2
    
# Defining the value of R0, the 0 index will have the value for redshift evolution merging rate, the 1 index would have the one for constant merging rate


SOBBH_R0= 28.3# +13.9 - 9.1 GPC⁻³ yr^⁻¹ Value of the merging rate fitted at z = 0.2


def SOBBH_R(z):
    return SOBBH_R0[0]*SOBBH_CorrRz*((1. + z)**SOBBH_k)/(1. + ((1. + z)/2.9)**(SOBBH_k + 2.9))
    
# If we wish to generate just a spike of events at a certain redshift range coming from a merging rate with fixed amplitude, we fix the following        
        

<h3> PBH - Merging Rates </h3>

We use the same model described by [S. S. Bavera et al](https://arxiv.org/pdf/2109.05836.pdf) for the redshift evolution of the merging rate. The amplitude of the perturbation can still be parametrized using the $fR$ approach as in the previous model :

In [13]:
PBH_R0 = 28.3 # +14.8 - 10.0 GPC⁻³ yr^⁻¹ Value of the merging rate fitted in at z = 0.2 in ligo population inference paper arxiv2111.03634
PBH_CorrfRz = SOBBH_CorrRz  # normalization factor needed to express the value of the LIGO merging rate in z=0
    
def PBH_fR(z,f):
    return f*PBH_R0*PBH_CorrfRz*((t_z(z)/t_0)**(-34./37.))


<h3> PBH - Gaussian Mass Distribution </h3>

We can define a Gaussian mass distribution for PBH as :

In [14]:
if PBH_Gaussian:
    PBH_m = 0. # Solar Masses. Minimum value assumed for the PBH mass
    PBH_M = 150. # Solar Masses. Maximum value assumed for the PBH mass
    PBH_massprec = 300 # Binning density for the mass range
    PBH_pdfmspan = np.linspace(0., 100., 100) # this span will be needed to compute the figures of merit
    PBH_sigmamspan = [1. ,5. ,10. ,15.] # Values of sigma_m to be spanned by the simulation
    
    # We use the following distribution for the mass, this tend to a monochromatic mass function for small values of sigma, yet it can be used to generalize the result to a wider subset of cases
    def PBH_MassGauss(m, PBH_mu, PBH_sigmam, PBH_GSnorm):
        return ((1./(PBH_sigmam*np.sqrt(2.*np.pi)))*np.exp(-0.5*((m-PBH_mu)/PBH_sigmam)**2.))*1./PBH_GSnorm
    
    # This function is to estimate the normalization constant
    def PBH_GaussPS(PBH_ranm, PBH_mu, PBH_sigmam):

        PBH_midm = 0.5*(PBH_ranm[1::] + PBH_ranm[:-1:])

        ris =  simpson(PBH_MassGauss(PBH_midm, PBH_mu, PBH_sigmam, 1.), PBH_midm)
            
        return ris


<h3> PBH - Log-Normal Mass Distribution </h3>

We can define a Log-Normal mass distribution for PBH as described in the paper by [S. S. Bavera et al ](https://arxiv.org/abs/2109.05836):

In [15]:
if PBH_LogNormal:
    # We use the following distribution for the mass
    PBH_m = 0. # Solar Masses. Minimum value assumed for the PBH mass
    PBH_M = 150. # Solar Masses. Maximum value assumed for the PBH mass
    PBH_massprec = 300 # Binning density for the mass range
    PBH_pdfmspan = np.linspace(0, 100., 100) # this span will be needed to compute the figures of merit
   
    PBH_sigmamnspan = [0.1 ,0.5 ,1. ,2.5] # Values of sigma_m to be spanned by the simulation
    
    def PBH_MassLNorm(m, PBH_Mc, PBH_sigmamn, PBH_LNnorm):
        return (1./(np.sqrt(2*np.pi)*PBH_sigmamn*m))*np.exp(-(np.log(m/PBH_Mc)**2)/(2*PBH_sigmamn**2))*1./PBH_LNnorm
    
    # This function is to estimate the normalization constant
    def PBH_LNnormPS(PBH_ranm, PBH_Mc, PBH_sigmamn):
        
        PBH_midm = 0.5*(PBH_ranm[1::] + PBH_ranm[:-1:])

        ris =  simpson(PBH_MassLNorm(PBH_midm, PBH_Mc, PBH_sigmamn, 1.), PBH_midm)
            
        return ris


<h3> Functions to estimate the $f_{PBH}$ given the value of $\varepsilon$</h3>

The function to minimize in order to find the value of $f_{PBH}$ given $\varepsilon$ is as follows

In [16]:
def f_resid(f_PBH, our_f, PBH_mu, PBH_sigma, PBH_norm, Avg_m, Avg_msqrd):
    if PBH_LogNormal:
        to_int = lambda y, x : ((x + y)**(-32./37.))*(EtaMass(x,y)**(-34./37.))*S1(f_PBH, x, y, Avg_m, Avg_msqrd)*PBH_MassLNorm(x, PBH_mu, PBH_sigma, PBH_norm)*PBH_MassLNorm(y, PBH_mu, PBH_sigma, PBH_norm)
    if PBH_Gaussian:
        to_int = lambda y, x : ((x + y)**(-32./37.))*(EtaMass(x,y)**(-34./37.))*S1(f_PBH, x, y, Avg_m, Avg_msqrd)*PBH_MassGauss(x, PBH_mu, PBH_sigma, PBH_norm)*PBH_MassGauss(y, PBH_mu, PBH_sigma, PBH_norm)
    return ((1.6e6/(PBH_R0*PBH_CorrfRz))*(f_PBH**(53./37.))*dblquad(to_int, PBH_midm[0], PBH_midm[-1], PBH_midm[0], PBH_midm[-1], epsrel=1.e-4)[0] - our_f)**2.

We can estimate this results over the maps by running

In [17]:
def FpbhToVarEpsilon(i, mat):        
        
    # Estimating the variables for the Gaussian Case 
        
    if PBH_Gaussian:
        PBH_mu = 0.5*(PBH_pdfmspan[i + 1] + PBH_pdfmspan[i])
        PBH_GSnorm = PBH_GaussPS(PBH_midm, PBH_mu, PBH_sigmam)
        Avg_m = simpson(PBH_midm*PBH_MassGauss(PBH_midm, PBH_mu, PBH_sigmam, PBH_GSnorm), PBH_midm)
        Avg_msqrd = simpson(PBH_midm*PBH_midm*PBH_MassGauss(PBH_midm, PBH_mu, PBH_sigmam, PBH_GSnorm), PBH_midm)
        
    # Estimating the variables for the LogNormal case
        
    if PBH_LogNormal:
        PBH_mu = 0.5*(PBH_pdfmspan[i + 1] + PBH_pdfmspan[i])
        PBH_LNnorm = PBH_LNnormPS(PBH_midm, PBH_mu, PBH_sigmamn)
        Avg_m = simpson(PBH_midm*PBH_MassLNorm(PBH_midm, PBH_mu, PBH_sigmamn, PBH_LNnorm), PBH_midm)
        Avg_msqrd = simpson(PBH_midm*PBH_midm*PBH_MassLNorm(PBH_midm, PBH_mu, PBH_sigmamn, PBH_LNnorm), PBH_midm)
       
    # Defining common variables
    
    frange_inv = PBH_frange[::-1]
    res_inv = frange_inv*0.
    Initial_guess = 0.005
    
    # Running the minimization
    
    for j in range(len(frange_inv)):
        our_f = frange_inv[j]
        if PBH_Gaussian:
            to_minimize = partial(f_resid, our_f = our_f, PBH_mu = PBH_mu, PBH_sigma = PBH_sigmam, PBH_norm = PBH_GSnorm, Avg_m = Avg_m, Avg_msqrd = Avg_msqrd)
        if PBH_LogNormal:
            to_minimize = partial(f_resid, our_f = our_f, PBH_mu = PBH_mu, PBH_sigma = PBH_sigmamn, PBH_norm = PBH_LNnorm, Avg_m = Avg_m, Avg_msqrd = Avg_msqrd)
        if j == 0:
            res_inv[j] = minimize(to_minimize, Initial_guess, method='Nelder-Mead', options={'xatol':1.e-5, 'fatol':1.e-5}).x
        else:
            res_inv[j] = minimize(to_minimize, res_inv[j-1], method='Nelder-Mead', options={'xatol':1.e-5, 'fatol':1.e-5}).x
     
    res = res_inv[::-1]
    
    # Saving the dataset
    
    for j in range(len(frange_inv)):
        delta_val = pd.DataFrame([[i, j, res[j]],], columns = ['idx_pdfm', 'idx_var_epsilon', 'val'])
        mat.append(delta_val)                       
        
    if ((i*10)%len(PBH_pdfmspan) == 0) :
        print('Percentage of completition : ',(i*100.)/(len(PBH_pdfmspan)), '%', flush=True)
        
                               

<h2> Setting of the analyzed phase space </h2>

The simulation will be spanned over the following range of variables :

In [18]:
# Initializing the mas range
PBH_ranm1 = np.linspace(PBH_m, PBH_M, PBH_massprec)
PBH_ranm2 = PBH_ranm1
PBH_midm = 0.5*(PBH_ranm1[1::] + PBH_ranm1[:-1:])
PBH_frange = np.logspace(-3.,0.,100)

<h2> Main body of the simulation </h2>

To create the conversion map for the considered _PBH_ subpopulation, we can now run as a function of the _Mass PDF_ parameters:

In [19]:


if PBH_LogNormal:
    
    #Summing on the PBH background contribution in the case of a LogNormal PDF
    print('-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~')
    
    manager = Manager()
    ConvMap = {}
    
    for i in range(len(PBH_sigmamnspan)):
        print('-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~')
        Conv_app = np.zeros((len(PBH_pdfmspan) - 1, len(PBH_frange))) + 10.
        PBH_sigmamn = PBH_sigmamnspan[i]
        d_par = manager.list()
        print('Now simulating the Conversion map for PBH (part ',i + 1,' of ',len(PBH_sigmamnspan),'), this can take some time !')
        
        if __name__ == '__main__':                                    
            # start the worker processes equals to n_jobs
            pool = Pool(n_jobs)
            pool.map(partial(FpbhToVarEpsilon, mat = d_par), range(len(PBH_pdfmspan)-1))
            pool.close()
            pool.join()

        
        for count in range(len(d_par)):
            Conv_app[int(d_par[count]['idx_pdfm'])][int(d_par[count]['idx_var_epsilon'])] = float(d_par[count]['val'])
           
        
        ConvMap[i] = Conv_app.transpose()
        print('-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~')

if PBH_Gaussian:
    
    #Summing on the PBH background contribution for the case of a Gaussian PDF with several sigma_m
    print('-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~')
    
    manager = Manager()
    ConvMap = {}
    
    for i in range(len(PBH_sigmamspan)):
        print('-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~')
        Conv_app = np.zeros((len(PBH_pdfmspan) - 1, len(PBH_frange))) + 10.
        PBH_sigmam = PBH_sigmamspan[i]
        d_par = manager.list()
        print('Now simulating the Integrated factor for PBH (part ',i + 1,' of ',len(PBH_sigmamspan),'), this can take some time !')
                   
        if __name__ == '__main__':                                    
            # start the worker processes equals to n_jobs
            pool = Pool(n_jobs)
            pool.map(partial(FpbhToVarEpsilon, mat = d_par), range(len(PBH_pdfmspan)-1))
            pool.close()
            pool.join()

        
        for count in range(len(d_par)):
            Conv_app[int(d_par[count]['idx_pdfm'])][int(d_par[count]['idx_var_epsilon'])] = float(d_par[count]['val'])
           
        
        ConvMap[i] = Conv_app.transpose()
        print('-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~')

-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
Now simulating the Integrated factor for PBH (part  1  of  4 ), this can take some time !


NameError: name 'PBH_GSnorm' is not defined

<h2> Saving the dataset </h2>

We can save the data using :

In [ ]:
# Saving the values of z at which the perturbation will overtake the fiducial models

if PBH_LogNormal:
    fname = 'ConvMapLNPDF.pickle'
if PBH_Gaussian:
    fname = 'ConvMapGSPDF.pickle'
    
file_to_write = open(fname, "wb")
pickle.dump(ConvMap, file_to_write)


<h3> Setting alarm to inform when simulation is over </h3>

In [ ]:
#file = 'Alarm-ringtone.mp3'
#os.system("mpg123 "+file)